In [5]:
import qiskit
import qiskit_algorithms
import qiskit_optimization

from qiskit_aer import AerSimulator

from qiskit.primitives import Sampler
from qiskit.primitives import BackendSamplerV2 
from qiskit.primitives import StatevectorSampler #takes 2 args but 3 are passed assuming Sampler :(
from qiskit_ibm_runtime import SamplerV2 #needs backend

from qiskit.circuit.library import QAOAAnsatz

from qiskit_algorithms import QAOA, SamplingVQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA

from qiskit_optimization.algorithms import MinimumEigenOptimizer, SolutionSample, OptimizationResultStatus
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import LinearEqualityToPenalty, LinearInequalityToPenalty, QuadraticProgramToQubo
from qiskit_optimization.translators import from_docplex_mp # creates QuadraticProgram from docplex model

from qiskit.visualization import plot_histogram

import networkx as nx
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams["figure.dpi"] = 1000
mpl.rcParams['savefig.dpi'] = 1000

import math
import time
import contextlib
import io
import os

from Models import docplex_BQO, docplex_RBQO, docplex_QUBO, docplex_RQUBO
from Penalties import tight_qubo_penalty, tight_rqubo_penalty, naive_qubo_penalty, naive_rqubo_penalty, interpolated_qubo_penalty, interpolated_rqubo_penalty
from Helper_Functions import *
from Plot_qaoa_histogram import plot_qaoa_histogram
from Run_qaoa_extract_samples import run_qaoa_extract_samples

# from qiskit_algorithms.utils import algorithm_globals
# algorithm_globals.random_seed = 123

import warnings
# Suppress specific DeprecationWarning
warnings.filterwarnings('ignore', category=DeprecationWarning)


print("qiskit version:",qiskit.__version__)
print("qiskit algorithms version:",qiskit_algorithms.__version__)
print("qiskit optimization version:",qiskit_optimization.__version__)

qiskit version: 1.3.2
qiskit algorithms version: 0.3.1
qiskit optimization version: 0.6.1


In [2]:
# parameters
backend = AerSimulator()
sampler = Sampler()
cvar_alpha = 1 # alpha for CVaR aggregation in (0,1], 1 for full distribution, 0 for only the best solution
shots = 1024/cvar_alpha
sampler.set_options(shots=shots, backend=backend)
optimizer = COBYLA()
reps = 1
initial_point = np.zeros(2 * reps)

# solver
qaoa = QAOA(
    sampler=sampler, 
    optimizer=optimizer, 
    reps=reps, 
    initial_point=initial_point,
    aggregation=cvar_alpha
    )

np_solver = NumPyMinimumEigensolver() # exact classical solver
# create minimum eigen optimizer based on solver used
qaoa_optimizer = MinimumEigenOptimizer(qaoa)
np_optimizer = MinimumEigenOptimizer(np_solver)

In [3]:
# Number of random graphs to generate
num_graphs = 10

# Problem parameters
K = 3
num_nodes = 5
edge_probability = 0.5
weighted = False
weight_range = 1

# We'll track the QAOA means for each model across random graph instances.
model_labels = ["QUBO (Tight)", "QUBO (Naive)", "RQUBO (Tight)", "RQUBO (Naive)"]
model_means = {label: [] for label in model_labels}
model_feasibility_probs = {label: [] for label in model_labels}

for i in range(num_graphs):
    print(f"Graph {i + 1}:")

    # 1) Generate a random graph
    G = generate_graph(num_nodes, edge_probability, weighted, weight_range)

    # 2) Build Docplex/QUBO/RQUBO models
    tight_penalty = tight_qubo_penalty(G, K)
    dp_qubo_tight = docplex_QUBO(G, K, tight_penalty, "Max-K-Cut")

    naive_penalty = naive_qubo_penalty(G, K)
    dp_qubo_naive = docplex_QUBO(G, K, naive_penalty, "Max-K-Cut")

    tight_penalty_r = tight_rqubo_penalty(G, K)
    dp_rqubo_tight = docplex_RQUBO(G, K, tight_penalty_r, "Max-K-Cut")

    naive_penalty_r = naive_rqubo_penalty(G, K)
    dp_rqubo_naive = docplex_RQUBO(G, K, naive_penalty_r, "Max-K-Cut")

    # 3) Run QAOA to extract raw samples for all models
    results_dict = run_qaoa_extract_samples(
        [dp_qubo_tight, dp_qubo_naive, dp_rqubo_tight, dp_rqubo_naive],
        ["QUBO (Tight)", "QUBO (Naive)", "RQUBO (Tight)", "RQUBO (Naive)"],
        optimizer=qaoa_optimizer,
    )

    # 4) Compute the mean (expected value) of each model's QAOA distribution
    #    and store it in model_means for later plotting.
    for label, result in results_dict.items():
        samples = result["samples"]
        filtered_samples, probability_feasible = feasibility_filter(G, K, samples, label)

        distribution_mean = sum(s.fval * s.probability for s in filtered_samples)
        model_means[label].append(distribution_mean)
        model_feasibility_probs[label].append(probability_feasible)

    # ----------------------------------------------------------------------
    # (Optional) Immediately plot histograms for this graph instance
    # ----------------------------------------------------------------------
    # Plot the histogram with default settings (no filtering, no re-penalization).
    # plot_qaoa_histogram(results_dict, 'QUBO (Tight)', 'QUBO (Naive)')
    # plot_qaoa_histogram(results_dict, 'RQUBO (Tight)', 'RQUBO (Naive)')

    # # Plot the histogram while filtering out infeasible solutions.
    # plot_qaoa_histogram(results_dict, 'QUBO (Tight)', 'QUBO (Naive)',
    #                     Graph=G, Partitions=K, filter_infeasible=True)
    # plot_qaoa_histogram(results_dict, 'RQUBO (Tight)', 'RQUBO (Naive)',
    #                     Graph=G, Partitions=K, filter_infeasible=True)

Graph 1:
Solving QUBO (Tight):
Classically calculating max objective value...
Max objective value for QUBO (Tight): 4.0
Running QAOA...
QAOA result: objective function value: 3.9999999999999996
variable values: x00=0.0, x01=0.0, x02=1.0, x10=0.0, x11=0.0, x12=1.0, x20=0.0, x21=1.0, x22=0.0, x30=1.0, x31=0.0, x32=0.0, x40=0.0, x41=0.0, x42=0.0
status: SUCCESS
Time taken: 24.2636 seconds
Solving QUBO (Naive):
Classically calculating max objective value...
Max objective value for QUBO (Naive): 4.0
Running QAOA...
QAOA result: objective function value: 4.0
variable values: x00=0.0, x01=0.0, x02=1.0, x10=0.0, x11=0.0, x12=1.0, x20=0.0, x21=1.0, x22=0.0, x30=1.0, x31=0.0, x32=0.0, x40=1.0, x41=0.0, x42=0.0
status: SUCCESS
Time taken: 20.0931 seconds
Solving RQUBO (Tight):
Classically calculating max objective value...
Max objective value for RQUBO (Tight): 4.0
Running QAOA...
QAOA result: objective function value: 4.0
variable values: x00=0.0, x01=1.0, x10=1.0, x11=0.0, x20=1.0, x21=0.0, x30

In [18]:
fig, axs = plt.subplots(2, 1, figsize=(8, 8))

# First subplot: Approximation Ratio
data_for_boxplot = [model_means[label] for label in model_labels]
axs[0].boxplot(data_for_boxplot, labels=model_labels)
axs[0].set_ylabel("Approximation Ratio")
axs[0].set_xticklabels(model_labels, rotation=30, ha='right')
axs[0].set_ylim(0, 1)

# Second subplot: Probability Feasible
data_for_boxplot = [model_feasibility_probs[label] for label in model_labels]
axs[1].boxplot(data_for_boxplot, labels=model_labels)
axs[1].set_ylabel("Probability Feasible")
axs[1].set_xticklabels(model_labels, rotation=30, ha='right')
axs[1].set_ylim(0, 1)

plt.tight_layout()
# Ensure the directory exists (even though you mentioned it does)
os.makedirs("Plots", exist_ok=True)
plt.savefig("Plots/boxplot.png")

In [7]:
print("median approximation ratios:")
print([(label, np.median(model_means[label])) for label in model_labels])
print()
print("median feasibility probabilities:")
print([(label, np.median(model_feasibility_probs[label])) for label in model_labels])

median approximation ratios:
[('QUBO (Tight)', 0.8062500000000004), ('QUBO (Naive)', 0.6727182539682539), ('RQUBO (Tight)', 0.7574148859109013), ('RQUBO (Naive)', 0.6870438200191424)]

median feasibility probabilities:
[('QUBO (Tight)', 0.03369140625), ('QUBO (Naive)', 0.02685546875), ('RQUBO (Tight)', 0.51318359375), ('RQUBO (Naive)', 0.37255859375)]
